# About Dataset

### **Context**
This Online Retail II dataset contains all the transactions occurring for a UK-based and registered, non-store online retail between 01.12.2009 and 09.12.2011. The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

The dataset is available on Kaggle (https://www.kaggle.com/datasets/kabilan45/online-retail-ii-datase) and on UCI Machine Learning Repository (https://archive.ics.uci.edu/dataset/502/online+retail+ii).

### **Content**
#### **Attribute Information:**

- `Invoice`: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- `StockCode`: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- `Description`: Product (item) name. Nominal.
- `Quantity`: The quantities of each product (item) per transaction. Numeric.
- `InvoiceDate`: Invoice date and time. Numeric. The day and time when a transaction was generated.
- `Price`: Unit price. Numeric. Product price per unit in sterling (Â£).
- `Customer ID`: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- `Country`: Country name. Nominal. The name of the country where a customer resides.
---

# 1. Getting to know our Dataset

In [1]:
# Import libraries
import numpy as np 
import pandas as pd 
import datetime as dt
import matplotlib.pyplot as plt 
from IPython.display import display
from collections import defaultdict

In [2]:
# Read excel's two sheets and merge them into a single Dataframe
df1_raw = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2009-2010")
df2_raw = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2010-2011")
df_raw = pd.concat([df1_raw, df2_raw])

In [3]:
# Review info on columns
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 73.3+ MB


In [4]:
# Review first rows of dataset
df_raw.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
# Shape of merged dataset
df_raw.shape

(1067371, 8)

It looks like we have **8 columns** with **1067371 rows**. 

### **1.1 Check for duplicates**


In [6]:
# Check for duplicates
print("Number of duplicate rows: ", df_raw.sort_values(by = "InvoiceDate").duplicated().sum())

Number of duplicate rows:  34335


In [7]:
# Visually inspect duplicate data records
_duplicated = df_raw[df_raw.duplicated(keep = False)].sort_values(by = ["InvoiceDate", "Invoice", "StockCode"])
_duplicated

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
379,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
365,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
363,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
...,...,...,...,...,...,...,...,...
541655,581538,23275,SET OF 3 HANGING OWLS OLLIE BEAK,1,2011-12-09 11:34:00,1.25,14446.0,United Kingdom
541656,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
541666,581538,23343,JUMBO BAG VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,2.08,14446.0,United Kingdom


It is fair to suspect that some of the **34335** duplicate records occurred due to merging data from the two excel sheets (*Years 2009-2010*, *Years 2010-2011*). We review the final date of the ***df1_raw*** and the first date of the ***df2_raw*** in order to confirm this assumption.

In [8]:
# Review the final date of the *df1_raw* and the first date of the *df2_raw*.   

df1_raw_final_date = df1_raw["InvoiceDate"].max()
df2_raw_start_date = df2_raw["InvoiceDate"].min()

print("Final date of the 'Year 2009-2010' dataset:", df1_raw_final_date)
print("Start date of the 'Year 2010-2011' dataset:", df2_raw_start_date)

Final date of the 'Year 2009-2010' dataset: 2010-12-09 20:01:00
Start date of the 'Year 2010-2011' dataset: 2010-12-01 08:26:00


It appears there is a data overlap of 9 days of transactions between the two excel sheets.
This can also be seen visually by filtering **df1_raw** by InvoiceDate, i.e. *InvoiceDate>=01.12.2010* and printing the first few rows of the resulted dataframe, while also printing the first few rows of **df2_raw**.

In [9]:
# First few rows from 12.2010 from the sheet 'Year 2009-2010'
df1_raw[df1_raw["InvoiceDate"] >= dt.datetime(2010,12,1)].sort_values(by = "InvoiceDate").head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
502938,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
502939,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
502940,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
502941,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
502942,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
# First rows from the sheet 'Year 2010-2011'
df2_raw.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


We remove the overlap in the dataset and create a new dataframe named **df**.

In [11]:
# Remove overlapped records, create df 
df = pd.concat([df1_raw[df1_raw["InvoiceDate"] < df2_raw_start_date], df2_raw])

In [12]:
df.shape

(1044848, 8)

In [13]:
 print("Number of duplicate rows:", df.sort_values(by = "InvoiceDate").duplicated().sum())

Number of duplicate rows: 11812


In [14]:
duplicated = df[df.duplicated(keep = False)].sort_values(by = ["InvoiceDate", "Invoice", "StockCode"])
duplicated

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
379,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
365,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
363,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
...,...,...,...,...,...,...,...,...
541655,581538,23275,SET OF 3 HANGING OWLS OLLIE BEAK,1,2011-12-09 11:34:00,1.25,14446.0,United Kingdom
541656,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
541666,581538,23343,JUMBO BAG VINTAGE CHRISTMAS,1,2011-12-09 11:34:00,2.08,14446.0,United Kingdom


In [15]:
# Export df to excel in order to load in PowerBI
# df.to_excel(r'C:\Users\argyr\OneDrive\Υπολογιστής\Diplvm\Giftware dataset\online_retail_II_removed duplicates.xlsx', index=False)

In [16]:
# List columns and see data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044848 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1044848 non-null  object        
 1   StockCode    1044848 non-null  object        
 2   Description  1040573 non-null  object        
 3   Quantity     1044848 non-null  int64         
 4   InvoiceDate  1044848 non-null  datetime64[ns]
 5   Price        1044848 non-null  float64       
 6   Customer ID  809561 non-null   float64       
 7   Country      1044848 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 71.7+ MB


In [17]:
# Take a closer look at column distributions
df.describe()

,Quantity,Price,Customer ID
count,1.044848e+06,1.044848e+06,809561.000000
mean,9.993649e+00,4.590546e+00,15319.760422
std,1.742185e+02,1.217042e+02,1695.812057
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13969.000000
50%,3.000000e+00,2.100000e+00,15248.000000
75%,1.000000e+01,4.130000e+00,16792.000000
max,8.099500e+04,3.897000e+04,18287.000000


### **1.2 Identify Missing Values**


In [18]:
# Identify missing values
missing = df.isnull().sum()
print(missing)

Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235287
Country             0
dtype: int64


In [19]:
# Percentage of missing values in each column
missing/len(df) * 100

Invoice         0.000000
StockCode       0.000000
Description     0.409150
Quantity        0.000000
InvoiceDate     0.000000
Price           0.000000
Customer ID    22.518778
Country         0.000000
dtype: float64

---
# 2. Exploratory Data Analysis

### 2.1 Invoice Column

In [20]:
# Convert column Invoice's data type to string
df["Invoice"] = df["Invoice"].astype(str)

In [21]:
# Unique number of invoices
len(df["Invoice"].unique())

53628

In [22]:
# Create a DataFrame with the records of cancelled invoices
df_cancelled_inv = df[df["Invoice"].str.contains("C")]
df_cancelled_inv.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia


In [23]:
# Review dimensions of cancelled invoices DataFrame
df_cancelled_inv.shape

(19165, 8)

The transactions that contain cancelled invoices (Invoice starts with "C") are **19165**. Out of these 19165 transactions, only 1 has positive value on Quantity column, while the other 19164 Invoices that start with "C" also have negative values on Quantity column.

In [24]:
# Invoices that start with "C" and Quantity < 0
df_cancelled_inv[df_cancelled_inv["Quantity"] < 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [25]:
# Invoices that start with "C" and Quantity > 0
df_cancelled_inv[df_cancelled_inv["Quantity"] > 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
76799,C496350,M,Manual,1,2010-02-01 08:24:00,373.57,NaN,United Kingdom


Moreover, there are some records where Invoice starts with "**A**". Let's try to figure out the meaning of these records.

In [26]:
# Invoices that start with "A"
df[df["Invoice"].str.contains("A")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [27]:
df[df["Invoice"].str.contains("A")].shape

(6, 8)

There are only 6 transactions of what appears to be adjustments regarding debts of the company. This explains the strongly negative values on **Price** column. There is only one exception regarding Invoice "A563185" with *Price = 11062.06* (positive value). Invoice *A563185* seems to be related with *A563186* and *A563187*, although they cannot be characterised as duplicates.

In [28]:
# Invoice starts with "C" and Customer ID is blank
df_cancelled_inv["Customer ID"].isnull().sum()

719

### 2.2 Quantity Column

In [29]:
# Number of records where Quantity < 0 
len(df[(df["Quantity"] < 0)])

22557

There are **22557** records with negative value on Quantity. These records are more than the cancelled invoices with negative value on Quantity (19164), which means that negative quantity can be linked to other kinds of transactions too, not only cancelled invoices/returns of products.

In [63]:
df["Quantity"].isnull().sum()

0

In [30]:
# Number of records where Quantity < 0 and Customer ID is blank
df[(df["Quantity"] < 0) & df["Customer ID"].isnull()]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.0,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.0,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.0,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom
536910,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom


In [31]:
# Number of records where Quantity < 0, Customer ID is blank and Invoice starts with "C"
df_cancelled_inv[(df_cancelled_inv["Quantity"] < 0) & df_cancelled_inv["Customer ID"].isnull()]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
5897,C489859,M,Manual,-1,2009-12-02 14:45:00,69.57,NaN,United Kingdom
5898,C489860,90029,NEW BAROQUE SMALL NECKLACE BLACK,-1,2009-12-02 14:49:00,8.50,NaN,United Kingdom
6376,C489881,21102,"CANDLE IN JAR COLLAGE , VANILLA",-31,2009-12-02 16:20:00,2.55,NaN,Nigeria
6377,C489881,72802A,ROSE SCENT CANDLE IN JEWELLED BOX,-16,2009-12-02 16:20:00,4.25,NaN,Nigeria
11553,C490307,POST,POSTAGE,-1,2009-12-04 14:49:00,8.97,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
492207,C578097,22112,CHOCOLATE HOT WATER BOTTLE,-48,2011-11-22 17:31:00,4.25,NaN,United Kingdom
514984,C579757,47469,ASSORTED SHAPES PHOTO CLIP SILVER,-24,2011-11-30 14:56:00,0.65,NaN,United Kingdom
516454,C579907,22169,FAMILY ALBUM WHITE PICTURE FRAME,-2,2011-12-01 08:48:00,7.65,NaN,EIRE
524601,C580604,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:35:00,11586.50,NaN,United Kingdom


### 2.3 Customer ID Column

In [32]:
# Number of records where Customer ID is blank 
len(df[(df["Customer ID"].isnull())])

235287

Customer ID column refers to the unique ID number given to each customer. There are **235287** missing values in this column, namely **22.5%** of the total number of transactions (1044848). This proportion is of great importance, and given the chance we would ask of the data collectors to explain this gap in the dataset. Given the fact that this is not possible, we assume that these customers had not registered to the online shop and thus, were never assigned a customer ID. 

In [33]:
# Number of unique customers 
unique_customers = len(df["Customer ID"].unique())
print("There are {} unique Customer IDs in the dataset.".format(unique_customers - 1))

There are 5942 unique Customer IDs in the dataset.


We substract 1, which represents the distinct count of null values in Customer ID column. 

In [34]:
# Drop missing values in Customer ID (235 287)
df_nnz_customerID = df.dropna(subset = ["Customer ID"])
print("For the customer segmentation analysis that will follow, we will drop missing values from " 
      "Customer ID column and thus the shape of the DataFrame will be reduced to {}.".format(df_nnz_customerID.shape))

For the customer segmentation analysis that will follow, we will drop missing values from Customer ID column and thus the shape of the DataFrame will be reduced to (809561, 8).


### 2.4 Country Column

In [35]:
# List of countries 
countries_list = df["Country"].unique()
number_of_countries = len(df["Country"].unique())
print("The number of countries in the dataset is {} and they are listed below:\n\n".format(number_of_countries), countries_list)

The number of countries in the dataset is 43 and they are listed below:

 ['United Kingdom' 'France' 'USA' 'Belgium' 'Australia' 'EIRE' 'Germany'
 'Portugal' 'Japan' 'Denmark' 'Nigeria' 'Netherlands' 'Poland' 'Spain'
 'Channel Islands' 'Italy' 'Cyprus' 'Greece' 'Norway' 'Austria' 'Sweden'
 'United Arab Emirates' 'Finland' 'Switzerland' 'Unspecified' 'Malta'
 'Bahrain' 'RSA' 'Bermuda' 'Hong Kong' 'Singapore' 'Thailand' 'Israel'
 'Lithuania' 'West Indies' 'Lebanon' 'Korea' 'Brazil' 'Canada' 'Iceland'
 'Saudi Arabia' 'Czech Republic' 'European Community']


In [36]:
# Number of records where country is "Unspecified"
len(df[df["Country"] == "Unspecified"])

756

In [37]:
# Top 10 countries by number of transactions
print(df["Country"].value_counts()[:10]/len(df) * 100)

United Kingdom    91.877766
EIRE               1.692974
Germany            1.661773
France             1.345555
Netherlands        0.491746
Spain              0.360435
Switzerland        0.304638
Belgium            0.297747
Portugal           0.243098
Australia          0.180983
Name: Country, dtype: float64


A large proportion of the customers buy from United Kingdom (**91.8%**), which is also the base of the online shop.

### 2.5 StockCode Column

In [38]:
# Number of unique StockCode items
number_of_StockCodes = pd.unique(df["StockCode"])
print("The number of unique StockCodes is", len(number_of_StockCodes))

The number of unique StockCodes is 5305


According to Python, the unique values of the column StockCode is **5305**.
In parallel with Python we conducted the same analysis using Microsoft Power BI, where we constructed a measure using the DAX expression *DISTINCTCOUNT* in order to count the unique values of the column. The result in Power BI is **5131**. We tried to figure out which of these results was the right one.
It's reasonable to assume that Python internally regards certain values as distinct whereas Power BI does not. Otherwise, this discrepancy wouldn't arise. In order to investigate whether this suspicion is correct, we must first level the playing field with regard to the values of the column. To this end, we cast all values as type string and subsequently turn all strings to upppercase.  In this way all values are comparable with each other.
After the execution of these commands, we see that the unique values of column StockCode are **5132**!

In [39]:
len(df['StockCode'].astype(str).str.upper().sort_values().unique())

5132

In order to find that one value that remains as a difference, we exported both lists of unique StockCodes to excel. The value *47503J* was spotted as written with two different ways, namely **"47503J "** and **"47503J"**. We used the command *str.strip()* in order to substract the blank space that caused this and finally reached the conclusion that the unique values of column StockCode are **5131**.

In [40]:
# df_2222 = pd.DataFrame(df['StockCode'].astype(str).str.upper().sort_values().unique())
# df_2222.to_excel(r'C:\Users\argyr\OneDrive\Υπολογιστής\Diplvm\Giftware dataset\online_retail_II_stockcodepy.xlsx', index=False)

In [41]:
(df['StockCode'].astype(str).str.upper().sort_values().unique() == '47503J').sum()

1

In [42]:
(df['StockCode'].astype(str).str.upper().sort_values().unique() == '47503J ').sum()

1

Using Power BI, we didn't have to perform all these steps to reach the right number of unique StockCodes because the command *DISTINCTCOUNTNOBLANK* counts the unique values of a column ignoring the fact that there may be blank spaces or uppercase/lowercase letters in value names. While here this has been helpful,it is not always the case. Each dataset is unique and it is absolutely necessary for the analyst to deeply understand its peculiarities before heading on with (and trusting?) any analysis tool.

We create a new DataFrame (**df2**) in order to store the values of *StockCode* in their updated version. From now on, **df2** is the base for our analysis.

In [43]:
# New version of df. This is the base version for the next steps of the analysis.
df['StockCode'] = df['StockCode'].astype(str).str.upper().str.strip()
df2 = df.copy()

In [44]:
unique_stockcodes = df['StockCode'].unique()
len(unique_stockcodes)

5131

#### Top 10 StockCodes

In [65]:
df2["StockCode"].isnull().sum()

0

In [45]:
# We would like to subtract records whose Price is zero.
# However Python and Power BI do not seem to agree on the representation of zero.
# Therefore in order to determine which products should be excluded from the top ten analysis,
# we will find out which products have positive price, which have negative, and then subtract
# these from the lot.

In [46]:
df_price_positive = df2[df2["Price"] > 0.0]
df_price_positive.shape

(1038819, 8)

In [47]:
df_price_negative = df2[df2["Price"] < 0.0]
df_price_negative.shape

(5, 8)

In [48]:
df_price_zero = df2[df2["Price"] == 0]
df_price_zero.shape

(6024, 8)

In [49]:
# The above three sizes agree with those extracted from Power BI.

In [50]:
# We concatenate the two DataFrames.
df_nnz_prices = pd.concat([df_price_positive, df_price_negative])

In [51]:
# We group by StockCode and sum the values of Quantity in order to review the ten most popular StockCodes of the online shop.
df_fixed_stockcodes = df_nnz_prices.copy()
most_preferred_products = df_fixed_stockcodes.groupby(["StockCode"])["Quantity"].sum().sort_values(ascending=False)[:10]
print(most_preferred_products)
most_preferred_products = pd.DataFrame(most_preferred_products)

StockCode
84077     104675
85099B     96137
21212      93862
85123A     91202
22197      87733
84879      79782
17003      69665
21977      56011
84991      53856
15036      43343
Name: Quantity, dtype: int64


In [52]:
top10_stockcodes = most_preferred_products.index.tolist()
print(top10_stockcodes)

['84077', '85099B', '21212', '85123A', '22197', '84879', '17003', '21977', '84991', '15036']


In [53]:
# Print the descriptions of the ten most popular StockCodes
for i in range(len(top10_stockcodes)):
    print(df[df["StockCode"] == top10_stockcodes[i]]["Description"][:1])

1309    WORLD WAR 2 GLIDERS ASSTD DESIGNS
Name: Description, dtype: object
161    JUMBO BAG RED WHITE SPOTTY 
Name: Description, dtype: object
246    PACK OF 72 RETRO SPOT CAKE CASES
Name: Description, dtype: object
102    WHITE HANGING HEART T-LIGHT HOLDER
Name: Description, dtype: object
550    POPCORN HOLDER , SMALL 
Name: Description, dtype: object
15    ASSORTED COLOUR BIRD ORNAMENT
Name: Description, dtype: object
2937    BROCADE RING PURSE 
Name: Description, dtype: object
210    PACK OF 60 PINK PAISLEY CAKE CASES
Name: Description, dtype: object
211    60 TEATIME FAIRY CAKE CASES
Name: Description, dtype: object
2922    ASSORTED COLOURS SILK FAN
Name: Description, dtype: object


### 2.6 InvoiceDate Column

The column **InvoiceDate** refers to the date and time that a transaction was generated. The transactions range from 01.12.2009 to 09.12.2011, as described at the start of the analysis.

In [54]:
# There are only 5 records whose price is negative. These transactions, as we have mentioned above,
# are purely logistical and do not reflect sales. 

In [55]:
# Our goal now is to find out which month has the most sales. 
# We restrict our analysis only to products whose price is positive.
df3 = df_price_positive.copy()

In [56]:
earliest_invoice_date = df3["InvoiceDate"].min()
latest_invoice_date = df3["InvoiceDate"].max()
date_fmt = "%d-%m-%Y"
print("The data ranges from {} to {}.".format(earliest_invoice_date.strftime(date_fmt), 
                                             latest_invoice_date.strftime(date_fmt) 
                                            )
     )

The data ranges from 01-12-2009 to 09-12-2011.


In [57]:
# Convert InvoiceDate column to datetime format
df3["InvoiceDate"] = pd.to_datetime(df3["InvoiceDate"])

# Create Month column
df3["Month"] = df3["InvoiceDate"].dt.strftime("%B")

# Create TotalAmount column
df3["TotalAmount"] = df3["Price"] * df3["Quantity"] 

In [58]:
# Monthly sales
monthly_sales = df3.groupby(["Month"])["TotalAmount"].sum().sort_values(ascending=False)
print(monthly_sales)

Month
November     2884410.892
October      2154798.890
December     1982490.140
September    1873338.053
March        1449115.841
June         1370909.730
August       1361580.970
May          1338656.340
July         1300568.261
January      1184033.152
April        1137381.913
February     1031154.076
Name: TotalAmount, dtype: float64


In [59]:
# Our goal now is to find out which hour of the day has the most distinct orders. 
# We restrict our analysis only to products whose price is positive.

# Create Hour column
df3["Hour"] = df3["InvoiceDate"].dt.strftime("%H")

In [60]:
# Number of orders 
hourly_invoices = df3.groupby(["Hour"])["Invoice"].nunique()
print(hourly_invoices)

Hour
06      22
07     110
08    1145
09    3230
10    5457
11    6014
12    7469
13    6672
14    5934
15    5381
16    3538
17    2014
18     721
19     560
20     102
21       1
Name: Invoice, dtype: int64
